In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
import math
import copy
import sklearn
import string
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import torchtext
from torchtext.data import get_tokenizer
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
/usr/local/lib/python3.10/dist-packages/torchtext/data/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [3]:
pip install sentence-transformers

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
# from google.colab import drive
# from google.colab import files
# drive.mount('/content/drive')


In [ ]:
# uploaded = files.upload()

In [4]:
#The emotions are classified into six categories: sadness (0), joy (1), love (2), anger (3), fear (4), and surprise (5).
data = pd.read_csv("/content/text.csv")

In [5]:
data.head()

,Unnamed: 0,text,label
0,0,i just feel really helpless and heavy hearted,4
1,1,ive enjoyed being able to slouch about relax a...,0
2,2,i gave up my internship with the dmrg and am f...,4
3,3,i dont know i feel so lost,0
4,4,i am a kindergarten teacher and i am thoroughl...,4


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 416809 entries, 0 to 416808
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  416809 non-null  int64 
 1   text        416809 non-null  object
 2   label       416809 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 9.5+ MB


In [7]:
data.isna().sum()

,0
Unnamed: 0,0
text,0
label,0


In [8]:
#The emotions are classified into six categories: sadness (0), joy (1), love (2), anger (3), fear (4), and surprise (5).
data['label'].value_counts()

,count
label,
1,141067
0,121187
3,57317
4,47712
2,34554
5,14972


In [9]:
stop_words = stopwords.words('english')
def clean_data(tweet) -> str:
  #Convert to Lower Case
  tweet = tweet.lower()

  #Remove Punctuation from the tweet
  tweet = tweet.translate(str.maketrans('', '', string.punctuation))

  #Remove stopwords from the tweet
  remove_stopwords = [word for word in tweet.split() if word not in stop_words]
  tweet = ' '.join(remove_stopwords)

  #Remove digits
  tweet = re.sub(r"\d",'',tweet)

  #remove extra space
  tweet = re.sub(r"\s+"," ",tweet)

  return tweet

In [10]:
data['model_input_text'] = data['text'].apply(clean_data)
data.head()

,Unnamed: 0,text,label,model_input_text
0,0,i just feel really helpless and heavy hearted,4,feel really helpless heavy hearted
1,1,ive enjoyed being able to slouch about relax a...,0,ive enjoyed able slouch relax unwind frankly n...
2,2,i gave up my internship with the dmrg and am f...,4,gave internship dmrg feeling distraught
3,3,i dont know i feel so lost,0,dont know feel lost
4,4,i am a kindergarten teacher and i am thoroughl...,4,kindergarten teacher thoroughly weary job take...


In [11]:
from sentence_transformers import SentenceTransformer
# Load a pre-trained model
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

def embedding(sentence):
  embeddings = model.encode(sentence, device = device)
  return embeddings
def to_tensor(arr):
  return torch.tensor(arr).to(device)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [12]:
data['embedded'] = data['model_input_text'].apply(embedding)
data.head()

,Unnamed: 0,text,label,model_input_text,embedded
0,0,i just feel really helpless and heavy hearted,4,feel really helpless heavy hearted,"[0.19374976, 0.332692, 0.078160934, -0.0284417..."
1,1,ive enjoyed being able to slouch about relax a...,0,ive enjoyed able slouch relax unwind frankly n...,"[0.08337724, -0.27273342, 0.649474, 0.06829055..."
2,2,i gave up my internship with the dmrg and am f...,4,gave internship dmrg feeling distraught,"[-0.20253323, 0.15025131, 0.6583267, 0.0836373..."
3,3,i dont know i feel so lost,0,dont know feel lost,"[0.40322068, -0.06682833, 0.3310123, 0.2095593..."
4,4,i am a kindergarten teacher and i am thoroughl...,4,kindergarten teacher thoroughly weary job take...,"[0.13750468, 0.27389866, 0.21386781, 0.4160004..."


In [13]:
data['embedding'] = data['embedded'].apply(to_tensor)
data.head()

,Unnamed: 0,text,label,model_input_text,embedded,embedding
0,0,i just feel really helpless and heavy hearted,4,feel really helpless heavy hearted,"[0.19374976, 0.332692, 0.078160934, -0.0284417...","[tensor(0.1937, device='cuda:0'), tensor(0.332..."
1,1,ive enjoyed being able to slouch about relax a...,0,ive enjoyed able slouch relax unwind frankly n...,"[0.08337724, -0.27273342, 0.649474, 0.06829055...","[tensor(0.0834, device='cuda:0'), tensor(-0.27..."
2,2,i gave up my internship with the dmrg and am f...,4,gave internship dmrg feeling distraught,"[-0.20253323, 0.15025131, 0.6583267, 0.0836373...","[tensor(-0.2025, device='cuda:0'), tensor(0.15..."
3,3,i dont know i feel so lost,0,dont know feel lost,"[0.40322068, -0.06682833, 0.3310123, 0.2095593...","[tensor(0.4032, device='cuda:0'), tensor(-0.06..."
4,4,i am a kindergarten teacher and i am thoroughl...,4,kindergarten teacher thoroughly weary job take...,"[0.13750468, 0.27389866, 0.21386781, 0.4160004...","[tensor(0.1375, device='cuda:0'), tensor(0.273..."


In [ ]:
#In order to define Position Encoding, we can call function get_positional_encoding function below with seq_len = Number of Sentences in each input. However in this task of classifying tweets
#We know that each input has exactly one sentence only. So positional encoding is redundant for this example since we are using sentence level embedding. So positional encoding step is skipped
#for this task. If we did a Word level embedding in this task, then we would have needed to call positional encoding in our example and then seq_len = Number of words. But that is not the case since
#we used Sentence transformer model which finds the embedding for sentences.

# def get_positional_encodings(seq_len=1, embedding_dim=384):
#     pos_enc = np.zeros((seq_len, embedding_dim))
#     for pos in range(seq_len):
#         for i in range(0, embedding_dim, 2):
#             pos_enc[pos, i] = np.sin(pos / (10000 ** ((2 * i)/embedding_dim)))
#             if i + 1 < embedding_dim:
#                 pos_enc[pos, i + 1] = np.cos(pos / (10000 ** ((2 * (i + 1))/embedding_dim)))
#     return pos_enc

# position_encoding = get_positional_encodings()

# def combined_embedding(embedding):
#   comb_embedding = embedding + position_encoding.squeeze(0)
#   return comb_embedding


# data['combined_embedded'] = data['embedded'].apply(combined_embedding)
# data.head()

In [14]:
data = data.drop(['embedded'],axis=1)
data.head()

,Unnamed: 0,text,label,model_input_text,embedding
0,0,i just feel really helpless and heavy hearted,4,feel really helpless heavy hearted,"[tensor(0.1937, device='cuda:0'), tensor(0.332..."
1,1,ive enjoyed being able to slouch about relax a...,0,ive enjoyed able slouch relax unwind frankly n...,"[tensor(0.0834, device='cuda:0'), tensor(-0.27..."
2,2,i gave up my internship with the dmrg and am f...,4,gave internship dmrg feeling distraught,"[tensor(-0.2025, device='cuda:0'), tensor(0.15..."
3,3,i dont know i feel so lost,0,dont know feel lost,"[tensor(0.4032, device='cuda:0'), tensor(-0.06..."
4,4,i am a kindergarten teacher and i am thoroughl...,4,kindergarten teacher thoroughly weary job take...,"[tensor(0.1375, device='cuda:0'), tensor(0.273..."


In [15]:
def convert_label_tensor(num):
  return torch.tensor(num).to(device)

In [16]:
data['output_labels'] = data['label'].apply(convert_label_tensor)
data.head()

,Unnamed: 0,text,label,model_input_text,embedding,output_labels
0,0,i just feel really helpless and heavy hearted,4,feel really helpless heavy hearted,"[tensor(0.1937, device='cuda:0'), tensor(0.332...","tensor(4, device='cuda:0')"
1,1,ive enjoyed being able to slouch about relax a...,0,ive enjoyed able slouch relax unwind frankly n...,"[tensor(0.0834, device='cuda:0'), tensor(-0.27...","tensor(0, device='cuda:0')"
2,2,i gave up my internship with the dmrg and am f...,4,gave internship dmrg feeling distraught,"[tensor(-0.2025, device='cuda:0'), tensor(0.15...","tensor(4, device='cuda:0')"
3,3,i dont know i feel so lost,0,dont know feel lost,"[tensor(0.4032, device='cuda:0'), tensor(-0.06...","tensor(0, device='cuda:0')"
4,4,i am a kindergarten teacher and i am thoroughl...,4,kindergarten teacher thoroughly weary job take...,"[tensor(0.1375, device='cuda:0'), tensor(0.273...","tensor(4, device='cuda:0')"


In [17]:
def assign_embedded_output(num):
  return output_embedding[num]

class Output_Embedding(nn.Module):
  def __init__(self,num_output_labels,input_dimension):
    super(Output_Embedding,self).__init__()
    self.out_embedding = nn.Embedding(num_output_labels,input_dimension,device=device)

  def forward(self,output_labels):
    return self.out_embedding(output_labels)

In [18]:
num_out_labels = 6  #(We have Output Labels ranging from 0-5)
input_dimension = 384  #(The size of embedded Input should be equal to size of embedded output)

output_embedding_layer = Output_Embedding(num_out_labels,input_dimension)
labels = torch.tensor([0,1,2,3,4,5]).to(device)

output_embedding = output_embedding_layer(labels)
data['output_embedding'] = data['output_labels'].apply(assign_embedded_output)
data.head()

,Unnamed: 0,text,label,model_input_text,embedding,output_labels,output_embedding
0,0,i just feel really helpless and heavy hearted,4,feel really helpless heavy hearted,"[tensor(0.1937, device='cuda:0'), tensor(0.332...","tensor(4, device='cuda:0')","[tensor(0.0796, device='cuda:0', grad_fn=<Unbi..."
1,1,ive enjoyed being able to slouch about relax a...,0,ive enjoyed able slouch relax unwind frankly n...,"[tensor(0.0834, device='cuda:0'), tensor(-0.27...","tensor(0, device='cuda:0')","[tensor(-2.0185, device='cuda:0', grad_fn=<Unb..."
2,2,i gave up my internship with the dmrg and am f...,4,gave internship dmrg feeling distraught,"[tensor(-0.2025, device='cuda:0'), tensor(0.15...","tensor(4, device='cuda:0')","[tensor(0.0796, device='cuda:0', grad_fn=<Unbi..."
3,3,i dont know i feel so lost,0,dont know feel lost,"[tensor(0.4032, device='cuda:0'), tensor(-0.06...","tensor(0, device='cuda:0')","[tensor(-2.0185, device='cuda:0', grad_fn=<Unb..."
4,4,i am a kindergarten teacher and i am thoroughl...,4,kindergarten teacher thoroughly weary job take...,"[tensor(0.1375, device='cuda:0'), tensor(0.273...","tensor(4, device='cuda:0')","[tensor(0.0796, device='cuda:0', grad_fn=<Unbi..."


In [19]:
data = data.drop(['output_labels'],axis=1)
data = data.drop(['text'],axis=1)
data = data.drop(['Unnamed: 0'],axis=1)
data.rename(columns={'embedding':'input_embedding','label':'output_label'},inplace=True)
data.head()

,output_label,model_input_text,input_embedding,output_embedding
0,4,feel really helpless heavy hearted,"[tensor(0.1937, device='cuda:0'), tensor(0.332...","[tensor(0.0796, device='cuda:0', grad_fn=<Unbi..."
1,0,ive enjoyed able slouch relax unwind frankly n...,"[tensor(0.0834, device='cuda:0'), tensor(-0.27...","[tensor(-2.0185, device='cuda:0', grad_fn=<Unb..."
2,4,gave internship dmrg feeling distraught,"[tensor(-0.2025, device='cuda:0'), tensor(0.15...","[tensor(0.0796, device='cuda:0', grad_fn=<Unbi..."
3,0,dont know feel lost,"[tensor(0.4032, device='cuda:0'), tensor(-0.06...","[tensor(-2.0185, device='cuda:0', grad_fn=<Unb..."
4,4,kindergarten teacher thoroughly weary job take...,"[tensor(0.1375, device='cuda:0'), tensor(0.273...","[tensor(0.0796, device='cuda:0', grad_fn=<Unbi..."


In [20]:
def create_2d_tensor_input(array):
  res = array.clone().detach()
  res = torch.tensor(res)
  res = res.unsqueeze(0)
  return res

data['2d_imbedded'] = data['input_embedding'].apply(create_2d_tensor_input)
data.head()

<ipython-input-20-280e2a2f2d7f>:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  res = torch.tensor(res)


,output_label,model_input_text,input_embedding,output_embedding,2d_imbedded
0,4,feel really helpless heavy hearted,"[tensor(0.1937, device='cuda:0'), tensor(0.332...","[tensor(0.0796, device='cuda:0', grad_fn=<Unbi...","[[tensor(0.1937, device='cuda:0'), tensor(0.33..."
1,0,ive enjoyed able slouch relax unwind frankly n...,"[tensor(0.0834, device='cuda:0'), tensor(-0.27...","[tensor(-2.0185, device='cuda:0', grad_fn=<Unb...","[[tensor(0.0834, device='cuda:0'), tensor(-0.2..."
2,4,gave internship dmrg feeling distraught,"[tensor(-0.2025, device='cuda:0'), tensor(0.15...","[tensor(0.0796, device='cuda:0', grad_fn=<Unbi...","[[tensor(-0.2025, device='cuda:0'), tensor(0.1..."
3,0,dont know feel lost,"[tensor(0.4032, device='cuda:0'), tensor(-0.06...","[tensor(-2.0185, device='cuda:0', grad_fn=<Unb...","[[tensor(0.4032, device='cuda:0'), tensor(-0.0..."
4,4,kindergarten teacher thoroughly weary job take...,"[tensor(0.1375, device='cuda:0'), tensor(0.273...","[tensor(0.0796, device='cuda:0', grad_fn=<Unbi...","[[tensor(0.1375, device='cuda:0'), tensor(0.27..."


In [21]:
def create_2d_tensor_output(array):
  res = array.clone().detach()
  res = torch.tensor(res)
  res = res.unsqueeze(0)
  return res

data['2d_imbedded_output'] = data['output_embedding'].apply(create_2d_tensor_output)
data.head()

<ipython-input-21-6a8d91943e69>:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  res = torch.tensor(res)


,output_label,model_input_text,input_embedding,output_embedding,2d_imbedded,2d_imbedded_output
0,4,feel really helpless heavy hearted,"[tensor(0.1937, device='cuda:0'), tensor(0.332...","[tensor(0.0796, device='cuda:0', grad_fn=<Unbi...","[[tensor(0.1937, device='cuda:0'), tensor(0.33...","[[tensor(0.0796, device='cuda:0'), tensor(-0.1..."
1,0,ive enjoyed able slouch relax unwind frankly n...,"[tensor(0.0834, device='cuda:0'), tensor(-0.27...","[tensor(-2.0185, device='cuda:0', grad_fn=<Unb...","[[tensor(0.0834, device='cuda:0'), tensor(-0.2...","[[tensor(-2.0185, device='cuda:0'), tensor(-0...."
2,4,gave internship dmrg feeling distraught,"[tensor(-0.2025, device='cuda:0'), tensor(0.15...","[tensor(0.0796, device='cuda:0', grad_fn=<Unbi...","[[tensor(-0.2025, device='cuda:0'), tensor(0.1...","[[tensor(0.0796, device='cuda:0'), tensor(-0.1..."
3,0,dont know feel lost,"[tensor(0.4032, device='cuda:0'), tensor(-0.06...","[tensor(-2.0185, device='cuda:0', grad_fn=<Unb...","[[tensor(0.4032, device='cuda:0'), tensor(-0.0...","[[tensor(-2.0185, device='cuda:0'), tensor(-0...."
4,4,kindergarten teacher thoroughly weary job take...,"[tensor(0.1375, device='cuda:0'), tensor(0.273...","[tensor(0.0796, device='cuda:0', grad_fn=<Unbi...","[[tensor(0.1375, device='cuda:0'), tensor(0.27...","[[tensor(0.0796, device='cuda:0'), tensor(-0.1..."


In [22]:
size_of_verification_data = 0.1
size_of_test_data = 0.2

from sklearn.model_selection import train_test_split

X = data['2d_imbedded']
y = data['2d_imbedded_output']

X_model, X_verify, y_model, y_verify = train_test_split(X,y,test_size=size_of_verification_data,shuffle=True)
X_train, X_test, y_train, y_test = train_test_split(X_model, y_model, test_size=size_of_test_data, shuffle=True)

print(f"Info of X_verify: {X_verify.info()} \n\n")
print(f"Info of X_train: {X_train.info()} \n\n")
print(f"Info of X_test: {X_test.info()} \n\n")

<class 'pandas.core.series.Series'>
Index: 41681 entries, 217364 to 123760
Series name: 2d_imbedded
Non-Null Count  Dtype 
--------------  ----- 
41681 non-null  object
dtypes: object(1)
memory usage: 651.3+ KB
Info of X_verify: None 


<class 'pandas.core.series.Series'>
Index: 300102 entries, 55484 to 159801
Series name: 2d_imbedded
Non-Null Count   Dtype 
--------------   ----- 
300102 non-null  object
dtypes: object(1)
memory usage: 4.6+ MB
Info of X_train: None 


<class 'pandas.core.series.Series'>
Index: 75026 entries, 113955 to 334022
Series name: 2d_imbedded
Non-Null Count  Dtype 
--------------  ----- 
75026 non-null  object
dtypes: object(1)
memory usage: 1.1+ MB
Info of X_test: None 




In [23]:
from torch.utils.data import DataLoader
BATCH_SIZE = 16


train_dataloader = DataLoader(list(zip(X_train,y_train)), shuffle=True, batch_size=BATCH_SIZE)
for X_batch, y_batch in train_dataloader:
    # print(X_batch, y_batch)
    break

test_dataloader = DataLoader(list(zip(X_test,y_test)), shuffle=False, batch_size=BATCH_SIZE)
for X_batch, y_batch in test_dataloader:
    # print(X_batch, y_batch)
    break

verify_dataloader = DataLoader(list(zip(X_verify,y_verify)), shuffle=False, batch_size=BATCH_SIZE)
for X_batch, y_batch in verify_dataloader:
    # print(X_batch, y_batch)
    break

In [24]:
print(f"Length of Train Dataloader is: {len(train_dataloader)}, Batches of Size: {BATCH_SIZE}")
print(f"Length of Test Dataloader is: {len(test_dataloader)},  Batches of Size: {BATCH_SIZE}")
print(f"Length of verify Dataloader is: {len(verify_dataloader)},  Batches of Size: {BATCH_SIZE}")

Length of Train Dataloader is: 18757, Batches of Size: 16
Length of Test Dataloader is: 4690,  Batches of Size: 16
Length of verify Dataloader is: 2606,  Batches of Size: 16


In [25]:
for X_batch,y_batch in train_dataloader:
  print(f"length of X_Batch: {len(X_batch)}")
  print(f"length of y_batch: {len(y_batch)}")
  print(f"lenght of each input sample in a batch: {len(X_batch[0])}")
  print(f"lenght of each output sample in a batch: {len(y_batch[0])}")
  break

length of X_Batch: 16
length of y_batch: 16
lenght of each input sample in a batch: 1
lenght of each output sample in a batch: 1


In [ ]:
# train_feature_batch, train_labels_batch = next(iter(train_dataloader))
# len(train_feature_batch), len(train_labels_batch)

In [26]:
class MultiHead_Attention(nn.Module):
  def __init__(self,input_dimension,num_heads,device):
    super(MultiHead_Attention,self).__init__()
    assert input_dimension % num_heads == 0, "Input dimensions must be a multiple of num_heads"

    self.input_dimension = input_dimension
    self.num_heads = num_heads
    self.single_head_dim = input_dimension // num_heads

    self.QMatrix = nn.Linear(input_dimension,input_dimension).to(device)       #Linear layer to hold the weight of Query Matrix  -> Projects input embeddings into query space
    self.KMatrix = nn.Linear(input_dimension,input_dimension).to(device)     #Linear layer to hold the weight of Key Matrix    -> Projects input embeddings into Key space
    self.VMatrix = nn.Linear(input_dimension,input_dimension).to(device)       #Linear layer to hold the weight of Value Matrix  -> Projects input embeddings into Value space
    self.OutMatrix = nn.Linear(input_dimension,input_dimension).to(device)     #Linear layer to hold the concatenated result of all the attention heads in original embedding size input_dimensions

  def create_attention_heads(self,input_embedding):
    """View method is called to reshape the input embedding matrix. After reshape new shape of tensor is (batch_size,seq_length,self.num_heads,self.single_head_dim)
    The transpose method is called after reshape to switch the dimensions at index 1 and 2 respectively."""
    batch_size, seq_length, d_model = input_embedding.size()
    #View Method breaks the input tensor into separate heads so that model can attend to different parts of the sequence simultaneously.
    #Transpose method is a necessary step to ensure that individual heads are alligned for matrix multiplication which helps to calculate attention scores.
    return input_embedding.view(batch_size,seq_length,self.num_heads,self.single_head_dim).transpose(1,2)

  def calculate_attention_scores(self,Q, K, V, mask = None):
    #Calculate the attention score belwo
    #Scale the attention score by the square root of the dimension of the keys (single_head_dim).
    #This helps to stabilize the gradients during training by preventing the scores from growing too large.
    attention_score = torch.matmul(Q,K.transpose(-2,-1)) / math.sqrt(self.single_head_dim)
    print(f"shape of attention score: {attention_score.shape}")
    # if mask is not None:
    #   mask = mask.unsqueeze(1)
    #   print(mask.shape)
    #   attention_score = attention_score.masked_fill(mask == 0,-1e9)
    #   V = V.view(V.size(-1)*4,-1)
    #Apply the softmax function to the attention scores along the last dimension (seq_length).
    #This converts the scores into probabilities that sum to 1 across the sequence length, allowing the model to attend to different positions in the input sequence.
    attention_prob = torch.softmax(attention_score,dim=-1)
    output = torch.matmul(attention_prob,V)
    return output

  def combine_all_heads(self,output):
    batch_size,number_of_heads,seq_length,output_embedding_dim = output.size()
    #Contiguous: Ensures the tensor memory is contiguous, which is necessary for the next view operation.
    #View: Reshape the tensor back to original shape as input
    return output.transpose(1,2).contiguous().view(batch_size,seq_length,self.input_dimension)

  def forward(self,Q,K,V,mask=None):
    Q = self.create_attention_heads(self.QMatrix(Q))
    K = self.create_attention_heads(self.KMatrix(K))
    V = self.create_attention_heads(self.VMatrix(V))

    attention_out = self.calculate_attention_scores(Q,K,V,mask)
    output = self.OutMatrix(self.combine_all_heads(attention_out))
    return output

In [27]:
class Test_MultiHead_Attention(nn.Module):
  def __init__(self,input_dimension,num_heads):
    super(Test_MultiHead_Attention,self).__init__()
    self.attention_block = MultiHead_Attention(input_dimension,num_heads,device=device)
  def forward(self,input_tensor):
    return self.attention_block(input_tensor,input_tensor,input_tensor)

In [28]:
output_attention_block_test = Test_MultiHead_Attention(384,4)
for X_batch, y_batch in train_dataloader:
    out_block = output_attention_block_test(X_batch)
    print(f"Size of Multhead Attention Block output: {out_block.shape}")
    print(f"Size of X_batch Block: {X_batch.shape}")
    print(out_block == X_batch)
    break

shape of attention score: torch.Size([16, 4, 1, 1])
Size of Multhead Attention Block output: torch.Size([16, 1, 384])
Size of X_batch Block: torch.Size([16, 1, 384])
tensor([[[False, False, False,  ..., False, False, False]],

        [[False, False, False,  ..., False, False, False]],

        [[False, False, False,  ..., False, False, False]],

        ...,

        [[False, False, False,  ..., False, False, False]],

        [[False, False, False,  ..., False, False, False]],

        [[False, False, False,  ..., False, False, False]]], device='cuda:0')


In [29]:
class FeedForward(nn.Module):
  def __init__(self,input_dimension,hidden_dimension,device):
    super(FeedForward,self).__init__()
    self.first_layer = nn.Linear(input_dimension,hidden_dimension).to(device)
    self.second_layer = nn.Linear(hidden_dimension,input_dimension).to(device)
    self.relu = nn.ReLU().to(device)

  def forward(self,input_tensor):
    return self.second_layer(self.relu(self.first_layer(input_tensor)))

In [30]:
class Test_FeedForward_Block(nn.Module):
  def __init__(self,input_dimension,hidden_dimension):
    super(Test_FeedForward_Block,self).__init__()
    self.feedforward_block = FeedForward(input_dimension,hidden_dimension,device=device)
  def forward(self,input_tensor):
    return self.feedforward_block(input_tensor)

In [31]:
feedforward_block_test = Test_FeedForward_Block(384,384*2)
for X_batch, y_batch in train_dataloader:
    out_block = feedforward_block_test(X_batch)
    print(f"Size of feedforward Block output: {out_block.shape}")
    print(f"Size of X_batch Block: {X_batch.shape}")
    print(out_block == X_batch)
    break

Size of feedforward Block output: torch.Size([16, 1, 384])
Size of X_batch Block: torch.Size([16, 1, 384])
tensor([[[False, False, False,  ..., False, False, False]],

        [[False, False, False,  ..., False, False, False]],

        [[False, False, False,  ..., False, False, False]],

        ...,

        [[False, False, False,  ..., False, False, False]],

        [[False, False, False,  ..., False, False, False]],

        [[False, False, False,  ..., False, False, False]]], device='cuda:0')


In [32]:
class Encoder_Unit(nn.Module):
  def __init__(self,input_dimension,num_of_heads,hidden_dimension,dropout_rate,device):
    super(Encoder_Unit,self).__init__()
    self.attention_block = MultiHead_Attention(input_dimension,num_of_heads,device)
    self.feed_forward_block = FeedForward(input_dimension,hidden_dimension,device)
    self.normblock_one = nn.LayerNorm(input_dimension).to(device)
    self.normblock_two = nn.LayerNorm(input_dimension).to(device)
    self.dropout_layer = nn.Dropout(dropout_rate).to(device)

  def forward(self,input_tensor,mask,device):
    #Shape of INput tensor: (Batch_Size,Length_of_input_sequence,input_dimension)
    attention_output = self.attention_block(input_tensor,input_tensor,input_tensor,mask)
    input_tensor = self.normblock_one(input_tensor + self.dropout_layer(attention_output)).to(device)
    feed_forward_output = self.feed_forward_block(input_tensor)
    input_tensor = self.normblock_two(input_tensor + self.dropout_layer(feed_forward_output)).to(device)
    return input_tensor

In [33]:
class Test_Encoder_Block(nn.Module):
  def __init__(self,input_dimension,num_of_heads,hidden_dimension,dropout_rate):
    super(Test_Encoder_Block,self).__init__()
    self.encoder_block = Encoder_Unit(input_dimension,num_of_heads,hidden_dimension,dropout_rate,device=device)

  def forward(self,input_tensor):
    return self.encoder_block(input_tensor,None,device=device)

In [34]:
encoder_block_test = Test_Encoder_Block(384,4,384*2,0.2)
for X_batch, y_batch in train_dataloader:
    out_block = encoder_block_test(X_batch)
    print(f"Size of encoder Block output: {out_block.shape}")
    print(f"Size of X_batch Block: {X_batch.shape}")
    print(out_block == X_batch)
    break

shape of attention score: torch.Size([16, 4, 1, 1])
Size of encoder Block output: torch.Size([16, 1, 384])
Size of X_batch Block: torch.Size([16, 1, 384])
tensor([[[False, False, False,  ..., False, False, False]],

        [[False, False, False,  ..., False, False, False]],

        [[False, False, False,  ..., False, False, False]],

        ...,

        [[False, False, False,  ..., False, False, False]],

        [[False, False, False,  ..., False, False, False]],

        [[False, False, False,  ..., False, False, False]]], device='cuda:0')


In [35]:
class Decoder_Unit(nn.Module):
  def __init__(self,input_dimension,num_of_heads,hidden_dimension,dropout_rate,device):
    super(Decoder_Unit,self).__init__()
    self.masked_attention_block = MultiHead_Attention(input_dimension,num_of_heads,device)
    self.cross_attention_block = MultiHead_Attention(input_dimension,num_of_heads,device)
    self.feed_forward_block = FeedForward(input_dimension,hidden_dimension,device)
    self.normblock_one = nn.LayerNorm(input_dimension).to(device)
    self.normblock_two = nn.LayerNorm(input_dimension).to(device)
    self.normblock_three = nn.LayerNorm(input_dimension).to(device)
    self.dropout_layer = nn.Dropout(dropout_rate).to(device)

  def forward(self,input_tensor,encoder_output,src_mask,target_mask):
    attention_output = self.masked_attention_block(input_tensor,input_tensor,input_tensor,target_mask)
    input_tensor = self.normblock_one(input_tensor + self.dropout_layer(attention_output)).to(device)
    attention_output = self.cross_attention_block(input_tensor,encoder_output,encoder_output,src_mask)
    input_tensor = self.normblock_two(input_tensor + self.dropout_layer(attention_output)).to(device)
    output = self.feed_forward_block(input_tensor).to(device)
    input_tensor = self.normblock_three(input_tensor + self.dropout_layer(output)).to(device)
    return input_tensor

In [36]:
class Test_Decoder_Block(nn.Module):
  def __init__(self,input_dimension,num_of_heads,hidden_dimension,dropout_rate):
    super(Test_Decoder_Block,self).__init__()
    self.decoder_block = Decoder_Unit(input_dimension,num_of_heads,hidden_dimension,dropout_rate,device=device)

  def generate_mask(self,source,target):
    source_mask = (source != 0).unsqueeze(1).unsqueeze(2)
    target_mask = (target != 0).unsqueeze(1).unsqueeze(3)
    seq_len = target.size(1)
    mask = (1-torch.triu(torch.ones(1,seq_len,seq_len),diagonal=1)).bool()
    target_mask = target_mask & mask
    return source_mask,target_mask

  def forward(self,input_tensor,encoder_output,source,target):
    # source_mask,target_mask = self.generate_mask(source,target)
    # return self.decoder_block(input_tensor,encoder_output,source_mask,target_mask)
    return self.decoder_block(input_tensor,encoder_output,source,target)

In [37]:
encoder_block_test = Test_Encoder_Block(384,4,384*2,0.2)
decoder_block_test = Test_Decoder_Block(384,4,384*2,0.2)
for X_batch, y_batch in train_dataloader:
    encoder_out_block = encoder_block_test(X_batch)
    print(f"Size of X_batch Block: {X_batch.shape}")
    target_in = y_batch[:,:-1]
    target_out = y_batch[:,1:]

    #decoder_out_block = decoder_block_test(y_batch,encoder_out_block,X_batch,target_out)
    #decoder_out_block = decoder_block_test(y_batch,encoder_out_block,X_batch,target_out)
    decoder_out_block = decoder_block_test(target_in,encoder_out_block,X_batch,target_out)
    print(f"Size of encoder Block output: {encoder_out_block.shape}")
    print(f"Size of decoder Block output: {decoder_out_block.shape}")
    print(f"Size of X_batch Block: {X_batch.shape}")
    print(f"Size of y_batch Block: {y_batch.shape}")
    print(decoder_out_block[0])
    break

shape of attention score: torch.Size([16, 4, 1, 1])
Size of X_batch Block: torch.Size([16, 1, 384])
shape of attention score: torch.Size([16, 4, 0, 0])
shape of attention score: torch.Size([16, 4, 0, 1])
Size of encoder Block output: torch.Size([16, 1, 384])
Size of decoder Block output: torch.Size([16, 0, 384])
Size of X_batch Block: torch.Size([16, 1, 384])
Size of y_batch Block: torch.Size([16, 1, 384])
tensor([], device='cuda:0', size=(0, 384), grad_fn=<SelectBackward0>)


In [ ]:
# class TransformerBlock(nn.Module):
#   def __init__(self,input_dimension,num_heads,num_layers,num_hidden_dim,dropout,num_output_labels,device):
#     super(TransformerBlock,self).__init__()
#     self.encoder_layers = nn.ModuleList([Encoder_Unit(input_dimension,num_heads,num_hidden_dim,dropout,device) for i in range(num_layers)])
#     self.decoder_layers = nn.ModuleList([Decoder_Unit(input_dimension,num_heads,num_hidden_dim,dropout,device) for i in range(num_layers)])
#     self.Output_layer = nn.Linear(input_dimension,num_output_labels).to(device)
#     self.dropout_layer = nn.Dropout(dropout).to(device)

#   def generate_mask(self,source,target):
#     source_mask = (source != 0).unsqueeze(1).unsqueeze(2)
#     target_mask = (target != 0).unsqueeze(1).unsqueeze(3)
#     seq_len = target.size(1)
#     mask = (1-torch.triu(torch.ones(1,seq_len,seq_len),diagonal=1)).bool()
#     target_mask = target_mask & mask
#     return source_mask,target_mask

#   def forward(self,source,target):
#     source_mask,target_mask = self.generate_mask(source,target)
#     source = self.dropout_layer(source)
#     target = self.dropout_layer(target)

#     encoder_output = source
#     for encoder_layer in self.encoder_layers:
#       encoder_output = encoder_layer(encoder_output,source_mask)

#     decoder_output = target
#     for decoder_layer in self.decoder_layers:
#       decoder_output = decoder_layer(decoder_output,encoder_output,source_mask,target_mask)

#     output = self.Output_layer(decoder_output)
#     return output

In [38]:
#IN the use case for sentiment analysis, I WOULD JUST NEED TO USE ENCODER ONLY TRansformer since I am trying to predict a single output based on each input

class TransformerBlock(nn.Module):
  def __init__(self,input_dimension,num_heads,num_layers,num_hidden_dim,dropout,num_output_labels,device):
    super(TransformerBlock,self).__init__()
    self.encoder_layers = nn.ModuleList([Encoder_Unit(input_dimension,num_heads,num_hidden_dim,dropout,device) for i in range(num_layers)])
    self.Output_layer = nn.Linear(input_dimension,num_output_labels).to(device)
    self.dropout_layer = nn.Dropout(dropout).to(device)

  def generate_mask(self,source,target):
    source_mask = (source != 0).unsqueeze(1).unsqueeze(2)
    target_mask = (target != 0).unsqueeze(1).unsqueeze(3)
    return source_mask,target_mask

  def forward(self,source,target):
    source_mask,target_mask = self.generate_mask(source,target)
    source = self.dropout_layer(source)
    target = self.dropout_layer(target)

    encoder_output = source
    for encoder_layer in self.encoder_layers:
      encoder_output = encoder_layer(encoder_output,source_mask,device)

    output = self.Output_layer(encoder_output).to(device)
    return output

In [39]:
class Test_Transformer_Block(nn.Module):
  def __init__(self,input_dimension,num_of_heads,num_layers,hidden_dimension,dropout_rate,num_output_labels):
    super(Test_Transformer_Block,self).__init__()
    self.trans_block_block = TransformerBlock(input_dimension,num_of_heads,num_layers,hidden_dimension,dropout_rate,num_output_labels,device=device)

  def forward(self,source,target):
    return self.trans_block_block(source,target)

In [40]:
transformer_block_test = Test_Transformer_Block(384,4, 4,384*2,0.2,6)
for X_batch, y_batch in train_dataloader:
    out_block = transformer_block_test(X_batch,y_batch)
    print(f"Size of encoder Block output: {out_block.shape}")
    print(f"Size of X_batch Block: {X_batch.shape}")
    print(f"{out_block[0]}")
    break

shape of attention score: torch.Size([16, 4, 1, 1])
shape of attention score: torch.Size([16, 4, 1, 1])
shape of attention score: torch.Size([16, 4, 1, 1])
shape of attention score: torch.Size([16, 4, 1, 1])
Size of encoder Block output: torch.Size([16, 1, 6])
Size of X_batch Block: torch.Size([16, 1, 384])
tensor([[ 0.3895, -0.3106,  0.3426,  0.3437,  1.0086,  0.0627]],
       device='cuda:0', grad_fn=<SelectBackward0>)
